# Imports

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.core.display import HTML
from IPython.display import Image
from scipy import stats
from sklearn.preprocessing import RobustScaler, MinMaxScaler# Start writing code here...

# Helper functions

In [ ]:
def jupyter_settings():
    %matplotlib inline
    %pylab inline
    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [25, 12]
    plt.rcParams['font.size'] = 24
    display( HTML( '<style>.container { width:100% !important; }</style>') )
    sns.set()
jupyter_settings()

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


# Load functions

In [ ]:
df5 = pd.read_csv('data/df4.csv')

In [ ]:
df5.head()

,id,gender,age,region_code,policy_sales_channel,previously_insured,annual_premium,vintage,vehicle_age,vehicle_damage,response
0,1,0,0.369231,28.0,26.0,0,0.585862,0.716263,3,1,1
1,2,0,0.861538,3.0,26.0,0,0.124508,0.598616,2,0,0
2,3,0,0.415385,28.0,26.0,0,0.441814,0.058824,3,1,1
3,4,0,0.015385,11.0,152.0,1,-0.203401,0.667820,1,0,0
4,5,1,0.138462,41.0,152.0,1,-0.278293,0.100346,1,0,0


In [ ]:
df5.dtypes

id                        int64
gender                    int64
age                     float64
region_code             float64
policy_sales_channel    float64
previously_insured        int64
annual_premium          float64
vintage                 float64
vehicle_age               int64
vehicle_damage            int64
response                  int64
dtype: object

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=3161d838-98a4-47ed-ae81-127ad2068af4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>